## Exporting a ResNet50 model to CoreML
To use a Fireball model in an iOS application, we can use [exportToCoreMl](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.exportToCoreMl) method. This notebook shows how to use this function to create a CoreML model ready to be deployed in an iOS app. It assumes that a trained ResNet50 model already exists in the ```Models``` directory. Please refer to the notebook [Image Classification with ResNet50](ResNet50.ipynb) for more info about using a pretrained ResNet50 model.

Fireball can also export models with reduced number of parameters, pruned models, and quatized models. Please refer to the following notebooks for more information:

- [Reducing number of parameters of ResNet50 Model](ResNet50-Reduce.ipynb)
- [Pruning ResNet50 Model](ResNet50-Prune.ipynb)
- [Quantizing ResNet50 Model](ResNet50-Quantize.ipynb)

Note: Fireball uses the [coremltools](https://github.com/apple/coremltools) python package to export CoreML models. 

## Load a pretrained model

In [1]:
from fireball import Model
from fireball.datasets.imagenet import ImageNetDSet
gpus='upto4'

# Create the test dataset for evaluation.
testDs = ImageNetDSet.makeDatasets('Test', batchSize=256, preProcessing='Crop256Cafe', numWorkers=8)

orgFileName = "Models/ResNet50RRPRQR.fbm"  # Reduced - Retrained - Pruned - Retrained - Quantized - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)   
model.initSession()
model.printLayersInfo()
results = model.evaluate()


Reading from "Models/ResNet50RRPRQR.fbm" ... Done.
Creating the fireball model "ResNet50" ... Done.
Metal device set to: Apple M1 Max

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 7 2 3               112 112 64    None                      7,114      
S1_L2_BN         112 112 64                             56 56 64      ReLU     MP(KSP):3 2 1    256        
S2_L1_RES2       56 56 64      2 Paths, 8 layers        56 56 256     ReLU                      48,512     
S2_L2_RES1       56 56 256     2 Paths, 7 layers        56 56 256     ReLU                      48,277     
S2_L3_RES1       56 56 256     2 Paths, 7 layers        56 56 256     ReLU                      48,808     


## Exporting to CoreML
CoreML handles the pre-processing of the images inside the model. The arguments ```rgbBias``` and ```scale``` are used to tell CoreML how to do this pre-processing. The pre-processed image is calculated by CoreML as:

```
processedImage = image * scale + rgbBias
```

For ResNet50, the processed images must be in BGR format with values normalized using the mean values 103.939, 116.779, 123.68 for blue, green, and red respectively. So, we are using the following values:
```
scale=1        # This is the default
rgbBias=[-123.68, -116.779, -103.939]
isBgr=True
```

In [2]:
import os

cmlFileName = orgFileName.replace('.fbm', '.mlmodel')

model.exportToCoreMl(cmlFileName, classNames=ImageNetDSet.classNames,
                     isBgr=True, rgbBias=[-123.68, -116.779, -103.939])

orgFileSize = os.stat("Models/ResNet50.fbm").st_size
print('Original Model File Size: {:,} bytes'.format(orgFileSize))
fileSize = os.stat(cmlFileName).st_size
print('CoreML Model File Size: {:,} bytes ({:2.2%} of original)'.format(fileSize, fileSize/orgFileSize))

scikit-learn version 1.4.0 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.



Exporting to CoreML model "Models/ResNet50RRPRQR.mlmodel" ... 
    Exported all 21 layers.                               
    Saving to "Models/ResNet50RRPRQR.mlmodel" ... Done.
Done (89.63 Sec.)
Original Model File Size: 102,562,787 bytes
CoreML Model File Size: 10,244,317 bytes (9.99% of original)


## Using netron to visualize the exported model
We can now visualize the model's network structure using the [netron](https://github.com/lutzroeder/netron) package.

In [3]:
import netron
import platform

if platform.system() == 'Darwin':      # Running on MAC
    netron.start(cmlFileName)   
else:
    import socket
    hostIp = socket.gethostbyname(socket.gethostname())
    netron.start(cmlFileName, address=(hostIp,8084))

Serving 'Models/ResNet50RRPRQR.mlmodel' at http://localhost:8080


## Running inference on the exported model
To verify the exported model, we can now run inference on it. Currently the CoreML runtime is only available on Mac. You can run all previous cells on a GPU-based machine and then copy the exported CoreML file to a Mac and test the model using the code in the following cell.

In [4]:
import platform
assert platform.system() == 'Darwin', "This is only supported when running on Mac!"

import cv2
from PIL import Image
import numpy as np
imageFileName = 'CoffeeMug.jpg'

img = cv2.imread(imageFileName)     # Reads image in BGR order

imgSize = img.shape[:2]
ratio = 256.0/min(imgSize)
newSize = (int(np.round(imgSize[1]*ratio)), int(np.round(imgSize[0]*ratio)))

# Note: INTER_AREA is best when shrinking and CV_INTER_CUBIC is best when enlarging
img = cv2.resize(img, newSize,  interpolation = (cv2.INTER_AREA if ratio<1.0 else cv2.INTER_CUBIC))

# Now crop the center 224x224 image
dw = newSize[0] - 224
dh = newSize[1] - 224
resizedImg = img[dh//2:dh//2+224, dw//2:dw//2+224,:]
pilImage = Image.fromarray( np.uint8(resizedImg[:,:,::-1]) ) # Convert to PIL image (RGB)

import coremltools
cmlFileName = "Models/ResNet50RRPRQR.mlmodel"
model = coremltools.models.MLModel(cmlFileName)
outputDict = model.predict({'InputImage': pilImage})

labels, probs = zip(*outputDict['ClassProbs'].items())
top3Idxs = np.argsort(probs)[-3:][::-1]    # Indexes of classes with 3 highest probs (decreasing order)
print('Top-3 Classes (For "%s"):'%(imageFileName))
for i in range(3):
    print('    %s (%f)'%(labels[top3Idxs[i]], probs[top3Idxs[i]])) 

Top-3 Classes (For "CoffeeMug.jpg"):
    coffee_mug (0.570312)
    cup (0.257080)
    pitcher (0.099182)


## Also look at

[Exporting ResNet50 Model to ONNX](ResNet50-ONNX.ipynb)

[Exporting ResNet50 Model to TensorFlow](ResNet50-TF.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with ResNet50](ResNet50.ipynb)

[Reducing number of parameters of ResNet50 Model](ResNet50-Reduce.ipynb)

[Pruning ResNet50 Model](ResNet50-Prune.ipynb)

[Quantizing ResNet50 Model](ResNet50-Quantize.ipynb)
